In [34]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
import folium
# intital loading map
map = folium.Map(location=[20.5937 ,78.9629 ], zoom_start=6, control_scale=True)
map

In [74]:
locationname = ['Mumbai' , 'Delhi' , 'Bangalore' ,'Kolkata' , 'Nellore']
locationname
totalnodes = len(locationname)

In [82]:
dict = {}
for i in range(len(locationname)):
    dict[locationname[i]] = i
start = dict['Mumbai']
end = dict['Bangalore']

In [83]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim


longitude = []
latitude = []


def findGeocode(city):
       
    try:

        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)  

for i in (locationname):

    if findGeocode(i) != None:
        loc = findGeocode(i)

        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)


In [84]:
# Create a matrix
matrix = np.array([latitude,longitude]).T
matrix


array([[19.0759899 , 72.8773928 ],
       [28.6517178 , 77.2219388 ],
       [12.9791198 , 77.5912997 ],
       [22.5414185 , 88.35769124],
       [14.4493717 , 79.9873763 ]])

In [85]:
#changing coordinates to distances in km
distances = []
temp =[]
for first in matrix:
    for second in matrix:
        if(first[0] == second[0] and first[1] == second[1]):
            temp.append(int(0))
        else:
            temp.append(geodesic(( first[0],first[1]), ( second[0],second[1])).km)

temp = np.reshape(temp, (len(latitude), len(latitude)))
distances = temp
pd.DataFrame(distances , columns = locationname)

,Mumbai,Delhi,Bangalore,Kolkata,Nellore
0,0.000000,1148.887036,842.248731,1655.629682,914.524520
1,1148.887036,0.000000,1735.689494,1306.478971,1598.286471
2,842.248731,1735.689494,0.000000,1555.437095,305.983709
3,1655.629682,1306.478971,1555.437095,0.000000,1257.678279
4,914.524520,1598.286471,305.983709,1257.678279,0.000000


In [86]:
def routefinder (node , visited , calculatedistance):
    
    if(len(visited) == totalnodes-1 and not(start in visited)):                  # base case
            visited.append(start);
            calculatedistance += distances[start][node]
            return visited , calculatedistance
        
        
    visited.append(node) 
    setofnodes = []
    returnvisited = []
    minvalue = 10000000000 #random big value
    for i in range(len(latitude)):
        if(i != start and not(i in visited)):
                setofnodes , distcheck = routefinder( i , visited , calculatedistance + distances[i][node])
                if(distcheck < minvalue):
                    returnvisited = setofnodes
                    minvalue = distcheck
    
    return returnvisited , minvalue
            
            
            
            
            

visited = []
nodes , caldistance = routefinder(end , visited , 0)
nodes

[]

In [87]:
temp =  100000000000
ans = []
def routefinder (node , visited , calculatedistance):
    global temp , ans;
    visited.append(node)
    
    if(len(visited) == totalnodes-1 and (start in visited) == False):
        visited.append(start)
        if(calculatedistance + distances[node][start] < temp):
            temp = calculatedistance + distances[node][start]
            ans = list(visited)

        
        visited.remove(start)
    
    
    for i in range(totalnodes):        
        if(i != start and not(i in visited)):
            routefinder( i , visited , calculatedistance + distances[i][node])
    
    visited.remove(node)         
            

routefinder(end , [] , 0)
temp

4019.027994704403

In [88]:
coordinates = []
for i in ans:
    coordinates.append([latitude[i] , longitude[i]])


coordinates

    


# Create the map and add the line
map1 = folium.Map(location=[latitude[start], longitude[start]], zoom_start=4)
my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
map1.add_child(my_PolyLine)


for i in range(len(latitude)):
    folium.Marker((latitude[i] , longitude[i]), popup=locationname[i]).add_to(map1)
map1